In [2]:
## ----setup, include=FALSE------------------------------------------------------------------------
knitr::opts_chunk$set(echo = TRUE, warnings = FALSE, purl = FALSE)
options(warn=-1)


## ----load.package--------------------------------------------------------------------------------
# library(devtools)
# devtools::install_github("JoshOBrien/gdalUtilities")
library(gdalUtilities)      # wrappers for GDAL utility programs that could be
                        #  called from the command line, but here via `sf`
# devtools::install_github("gearslaboratory/gdalUtils")
library(gdalUtils)      # wrappers for GDAL utility programs that could be
                        #  called from the command line,
library(sf)             # spatial data types -- Simple Features
library(stars)          # Spatiotemporal Arrays, Raster and Vector Data Cubes
library(dplyr)          # tidyverse data manipulation functions
library(ggplot2)        # gpplot graphics
library(maps)           # optional -- for boundary polygons
library(mapdata)



In [3]:
## ------------------------------------------------------------------------------------------------
drivers <- sf::st_drivers()
# print(drivers)
ix <- grep("GPKG", drivers$name,  fixed=TRUE)
drivers[ix,]
ix <- grep("ESRI", drivers$name,  fixed=TRUE)
drivers[ix,]
ix <- grep("CSV", drivers$name,  fixed=TRUE)
drivers[ix,]

,name,long_name,write,copy,is_raster,is_vector,vsi
,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
GPKG,GPKG,GeoPackage,TRUE,TRUE,TRUE,TRUE,TRUE


,name,long_name,write,copy,is_raster,is_vector,vsi
,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
ESRIC,ESRIC,Esri Compact Cache,FALSE,FALSE,TRUE,TRUE,TRUE
ESRI Shapefile,ESRI Shapefile,ESRI Shapefile,TRUE,FALSE,FALSE,TRUE,TRUE
ESRIJSON,ESRIJSON,ESRIJSON,FALSE,FALSE,FALSE,TRUE,TRUE


,name,long_name,write,copy,is_raster,is_vector,vsi
,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
CSV,CSV,Comma Separated Value (.csv),TRUE,FALSE,FALSE,TRUE,TRUE


In [9]:


## ----specify.wfs.address-------------------------------------------------------------------------
wfs <- "WFS:https://maps.isric.org/mapserv?map=/map/wosis_latest.map"


## ----ogrinfo.wfs---------------------------------------------------------------------------------
(layers.info <- st_layers(wfs))


## ----ogrinfo.site--------------------------------------------------------------------------------
profiles.info <-
  gdalUtils::ogrinfo(wfs, layer = "ms:wosis_latest_profiles",
                     ro = TRUE, so = TRUE, q = TRUE)
cat(profiles.info, sep="\n")


Driver: WFS 
Available layers:
                 layer_name geometry_type features fields crs_name
1    ms:wosis_latest_bdfi33                  77976     18   WGS 84
2    ms:wosis_latest_bdfiad                  14483     18   WGS 84
3    ms:wosis_latest_bdfifm                  14075     18   WGS 84
4    ms:wosis_latest_bdfiod                 131560     18   WGS 84
5    ms:wosis_latest_bdwsod                  75386     18   WGS 84
6    ms:wosis_latest_cecph7                 320354     18   WGS 84
7    ms:wosis_latest_cecph8                  25100     18   WGS 84
8      ms:wosis_latest_cfgr                 202399     18   WGS 84
9      ms:wosis_latest_cfvo                 246528     18   WGS 84
10     ms:wosis_latest_clay                 651970     18   WGS 84
11     ms:wosis_latest_ecec                 143381     18   WGS 84
12   ms:wosis_latest_elco20                  44350     18   WGS 84
13   ms:wosis_latest_elco25                  17736     18   WGS 84
14   ms:wosis_latest_elco50    

Metadata:
  ABSTRACT=Title: wosis_latest
  PROVIDER_NAME=ISRIC - World Soil Reference
  TITLE=Title: wosis_latest

Layer name: ms:wosis_latest_profiles
Metadata:
  ABSTRACT=Provides key information for all profiles (or point data) in WoSIS latest, including soil classification according to specified edition (year) of the World Reference Base for Soil Resources (WRB, up to qualifier level); FAO-Unesco Legend (up to soil unit level); USDA Soil Taxonomy (up to subgroup level).
       
WoSIS_latest is a "dynamic dataset" that contains the most recent complement of quality-assessed and standardised soil data served from WoSIS (ISRIC World Soil Information Service). The source data were shared by a wide range of data providers (see: https://www.isric.org/explore/wosis/wosis-contributing-institutions-and-experts).

Being dynamic, the contents of "wosis_latest" will change once new point data are acquired, cleansed and standardised, additional soil properties are considered, and/or when possib

In [6]:

## ----ogrinfo.eu.profiles.1-----------------------------------------------------------------------
central.eu.profiles.info <-
  gdalUtils::ogrinfo(wfs, ro=TRUE, so=TRUE, q=FALSE,
                     layer="ms:wosis_latest_profiles",
                     spat=c(6, 48, 8, 50),
                     verbose = FALSE)
head(central.eu.profiles.info, 8)


[1] "INFO: Open of `WFS:https://maps.isric.org/mapserv?map=/map/wosis_latest.map'"
[2] "      using driver `WFS' successful."                                        
[3] "Metadata:"                                                                   
[4] "  ABSTRACT=Title: wosis_latest"                                              
[5] "  PROVIDER_NAME=ISRIC - World Soil Reference"                                
[6] "  TITLE=Title: wosis_latest"                                                 
[7] ""                                                                            
[8] "Layer name: ms:wosis_latest_profiles"

In [10]:

## ----ogrinfo.eu.profiles.2-----------------------------------------------------------------------
ix.f <- grep("Feature Count", central.eu.profiles.info)
central.eu.profiles.info[ix.f]
ix.e <- grep("Extent", central.eu.profiles.info)
central.eu.profiles.info[ix.e]
ix.g <- grep("GEOGCRS", central.eu.profiles.info)
cat(paste(central.eu.profiles.info[ix.g:(ix.g+17)], collapse="\n"))

[1] "Feature Count: 327"

[1] "Extent: (6.004098, 48.018583) - (7.994063, 50.000000)"

GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],

In [11]:

## ----show.fields---------------------------------------------------------------------------------
ix.p <- grep("Geometry Column", central.eu.profiles.info)
n <- length(central.eu.profiles.info)
central.eu.profiles.info[ix.p+1:n]


## ----ogrinfo.india.profiles----------------------------------------------------------------------
india.profiles.info <-
  ogrinfo(wfs, ro=TRUE, so=TRUE, q=FALSE,
          layer="ms:wosis_latest_profiles",
          where="country_name='India'")



[1] "gml_id: String (0.0) NOT NULL"              
 [2] "profile_id: Integer (0.0)"                  
 [3] "profile_code: String (0.0)"                 
 [4] "dataset_code: String (0.0)"                 
 [5] "year: Integer (0.0)"                        
 [6] "month: Integer (0.0)"                       
 [7] "day: Integer (0.0)"                         
 [8] "site_id: Integer (0.0)"                     
 [9] "positional_uncertainty: String (0.0)"       
[10] "country_name: String (0.0)"                 
[11] "region: String (0.0)"                       
[12] "continent: String (0.0)"                    
[13] "longitude: Real (0.0)"                      
[14] "latitude: Real (0.0)"                       
[15] "wrb_reference_soil_group_code: String (0.0)"
[16] "wrb_reference_soil_group: String (0.0)"     
[17] "wrb_prefix_qualifiers: String (0.0)"        
[18] "wrb_suffix_qualifiers: String (0.0)"        
[19] "wrb_principal_qualifiers: String (0.0)"     
[20] "wrb_supplementary_qualifiers: String (0.0)" 
[21] "wrb_publication_year: Integer (0.0)"        
[22] "fao_major_group_code: String (0.0)"         
[23] "fao_major_group: String (0.0)"              
[24] "fao_soil_unit_code: String (0.0)"           
[25] "fao_soil_unit: String (0.0)"                
[26] "fao_publication_year: Integer (0.0)"        
[27] "usda_order_name: String (0.0)"              
[28] "usda_suborder: String (0.0)"                
[29] "usda_subgroup: String (0.0)"                
[30] "usda_great_group: String (0.0)"             
[31] "usda_publication_year: Integer (0.0)"       
[32] NA                                           
[33] NA                                           
[34] NA                                           
[35] NA                                           
[36] NA                                           
[37] NA                                           
[38] NA                                           
[39] NA                                           
[40] NA                                           
[41] NA                                           
[42] NA                                           
[43] NA                                           
[44] NA                                           
[45] NA                                           
[46] NA                                           
[47] NA                                           
[48] NA                                           
[49] NA                                           
[50] NA                                           
[51] NA                                           
[52] NA                                           
[53] NA                                           
[54] NA                                           
[55] NA                                           
[56] NA                                           
[57] NA                                           
[58] NA                                           
[59] NA                                           
[60] NA                                           
[61] NA                                           
[62] NA                                           
[63] NA                                           
[64] NA                                           
[65] NA                                           
[66] NA                                           
[67] NA                                           
[68] NA                                           
[69] NA                                           
[70] NA                                           
[71] NA                                           
[72] NA                                           
[73] NA                                           
[74] NA                                           
[75] NA                                           
[76] NA                                           
[77] NA                                           
[78] NA                                           
[79] NA                

In [12]:

## ----ogrinfo.india.profiles.2--------------------------------------------------------------------
ix.f <- grep("Feature Count", india.profiles.info)
india.profiles.info[ix.f]
ix.e <- grep("Extent", india.profiles.info)
india.profiles.info[ix.e]


[1] "Feature Count: 221"

[1] "Extent: (69.800000, 8.483333) - (94.050000, 32.000000)"

In [ ]:

## ----ogrinfo.properties, warnings=-1-------------------------------------------------------------
property.info <- gdalUtils::ogrinfo(wfs, layer = "ms:wosis_latest_bdfi33",
                                    ro = TRUE, so = TRUE, q = FALSE)
cat(property.info, sep="\n")
ix.f <- grep("Feature Count", property.info)
property.info[ix.f]


## ----ogrinfo.bd.india----------------------------------------------------------------------------
bd.india.info <- gdalUtils::ogrinfo(wfs, layer="ms:wosis_latest_bdfi33",
                                    where="country_name='India' AND upper_depth > 100",
                                    ro=TRUE, so=TRUE, q=FALSE)
ix.f <- grep("Feature Count", bd.india.info)
bd.india.info[ix.f]
(n.records <- as.numeric(strsplit(bd.india.info[ix.f],
                                  split=": ", fixed=TRUE)[[1]][2]))



In [ ]:

## ----setup.local.dir-----------------------------------------------------------------------------
wosis.dir.name <- "./wosis_latest"
if (!file.exists(wosis.dir.name)) dir.create(wosis.dir.name)



In [ ]:

## ----download.profile.gpkg-----------------------------------------------------------------------
layer.name <- "wosis_latest_profiles"
(dst.target.name <- paste0(wosis.dir.name,"/", layer.name, ".gpkg"))
if (!file.exists(dst.target.name)) { 
system.time(
  gdalUtilities::ogr2ogr(src=wfs,
          dst=dst.target.name,
          layer=layer.name,
          f = "GPKG",
          overwrite=TRUE,
          skipfailures=TRUE)
)
}
file.info(dst.target.name)$size/1024/1024


In [ ]:

## ----read.profile.gpkg---------------------------------------------------------------------------
profiles.gpkg <- st_read(dst.target.name)
class(profiles.gpkg)
dim(profiles.gpkg)
names(profiles.gpkg)


## ------------------------------------------------------------------------------------------------
head(sort(table(profiles.gpkg$country_name), decreasing=TRUE))


## ----map.profiles.gpkg, fig.width=12, fig.height=12----------------------------------------------
ggplot(data=profiles.gpkg[(profiles.gpkg$country_name=="Brazil"), ]) +
  aes(col=dataset_id) +
  geom_sf(shape=21, size=0.8, fill="black")



In [ ]:
## ----download.profiles---------------------------------------------------------------------------
layer.name <- "ms_wosis_latest_profiles"
(dst.target.name <- paste0(wosis.dir.name,"/", layer.name, ".shp"))
if (!file.exists(dst.target.name)) { 
system.time(
  gdalUtilities::ogr2ogr(src=wfs,
          dst=wosis.dir.name,
          layer=layer.name,
          f = "ESRI Shapefile",
          overwrite=TRUE,
          skipfailures=TRUE)
)
}
file.info(dst.target.name)$size/1024/1024



In [ ]:

## ----download.profiles.india---------------------------------------------------------------------
wosis.dir.name.india <- "./wosis_latest/india"
if (!file.exists(wosis.dir.name.india)) dir.create(wosis.dir.name.india)
layer.name <- "ms_wosis_latest_profiles"
(dst.target.name <- paste0(wosis.dir.name,"/", layer.name, ".shp"))
if (!file.exists(dst.target.name)) { 
  system.time(
  gdalUtilities::ogr2ogr(src=wfs,
          dst=wosis.dir.name.india,
          layer=layer.name,
          f = "ESRI Shapefile",
          where="country_name='India'",
          overwrite=TRUE,
          skipfailures=TRUE)
)
}
file.info(dst.target.name)$size/1024/1024

In [ ]:

## ----import.profiles-----------------------------------------------------------------------------
layer.name <- "ms_wosis_latest_profiles"
profiles <- sf::st_read(dsn=wosis.dir.name, layer=layer.name,
                stringsAsFactors = FALSE)
class(profiles)
dim(profiles)
names(profiles)
head(profiles)


## ----import.profiles.india-----------------------------------------------------------------------
profiles.india <- sf::st_read(dsn=wosis.dir.name.india, layer=layer.name,
                stringsAsFactors = FALSE)
dim(profiles.india)
names(profiles.india)
head(profiles.india)




In [ ]:
## ----download.bd---------------------------------------------------------------------------------
layer.name <- "ms_wosis_latest_bdfi33"
(dst.target.name <- paste0(wosis.dir.name,"/", layer.name, ".shp"))
if (!file.exists(dst.target.name)) { 
  system.time(
  gdalUtilities::ogr2ogr(src=wfs,
          dst=wosis.dir.name,
          layer=layer.name,
          f = "ESRI Shapefile",
          overwrite=TRUE,
          skipfailures=TRUE)
)
}
file.info(dst.target.name)$size/1024/1024


## ----download.bd.india---------------------------------------------------------------------------
layer.name <- "ms_wosis_latest_bdfi33"
(dst.target.name <- paste0(wosis.dir.name.india,"/", layer.name, ".shp"))
if (!file.exists(dst.target.name)) { 
  system.time(
    gdalUtilities::ogr2ogr(src=wfs,
                           dst=wosis.dir.name.india,
                           layer=layer.name,
                           f = "ESRI Shapefile",
                           where="country_name='India'",
                           overwrite=TRUE,
                           skipfailures=TRUE)
  )
}
file.info(dst.target.name)$size/1024/1024


## ----import.bd-----------------------------------------------------------------------------------
# here strings are just that, not to be interpreted as R factors
layer.name <- "ms_wosis_latest_bdfi33"
bd33 <- st_read(dsn=wosis.dir.name, layer=layer.name,
                stringsAsFactors = FALSE)
class(bd33)
names(bd33)
head(bd33)


## ----head.bd-------------------------------------------------------------------------------------
head(bd33$bdfi33_val)


## ----example.bd----------------------------------------------------------------------------------
bd33[75:80, c("profile_id","upper_dept","lower_dept","bdfi33_val","bdfi33_v_1")]


## ----import.bd.india, out.width='0.7\\linewidth'-------------------------------------------------
layer.name <- "ms_wosis_latest_bdfi33"
bd33.india <- st_read(dsn=wosis.dir.name.india, layer=layer.name,
                stringsAsFactors = FALSE)
class(bd33.india)
dim(bd33.india)
(profile.id.india <- unique(bd33.india$profile_id))
names(bd33.india)
hist(bd33.india$bdfi33_v_1, main="Bulk density, soil layers in India", 
     xlab="g cm^{-3}")
rug(bd33.india$bdfi33_v_1)



In [ ]:

## ----select.bd33.30cm----------------------------------------------------------------------------
bd30cm <- bd33.india %>%
  select(profile_id, upper_dept, lower_dept, bdfi33_v_1) %>%
  filter((lower_dept > 30) ) %>%
  group_by(profile_id) %>%    # may be more than one layer deeper than this
  arrange(upper_dept) %>%     # select the shallowest
  filter(row_number() == 1) %>%
  ungroup()
glimpse(bd30cm)


## ----join.profile.bd33---------------------------------------------------------------------------
india.pts <- profiles.india %>%
  select(profile_id, geometry) 
bd30cm.j <- 
  left_join(india.pts, st_drop_geometry(bd30cm), by="profile_id")
print(bd30cm.j)


## ------------------------------------------------------------------------------------------------
IN <- map_data(map = 'world',
               region = 'India')
# remove named subregions, i.e., the islands
ix <- which(is.na(IN$subregion))
IN <- IN[ix,]
ggplot() +
  geom_path(aes(x = long, y = lat), data=IN) +
  geom_sf(data = bd30cm, aes(col = bdfi33_v_1)) +
  labs(title = "Bulk density at 30 cm depth", col = expression(paste(g," ", cm)^-3))


In [ ]:


## ----download.profiles.csv-----------------------------------------------------------------------
wosis.dir.name.ceu <- "./wosis_latest/central_europe"
if (!file.exists(wosis.dir.name.ceu)) dir.create(wosis.dir.name.ceu)
src.layer.name <- "ms:wosis_latest_profiles"
dst.layer.name <- "wosis_latest_profiles_ceu"
(dst.target.name <- paste0(wosis.dir.name.ceu,"/",dst.layer.name,".csv"))
if (!file.exists(dst.target.name)) {
gdalUtilities::ogr2ogr(src=wfs, 
        dst=dst.target.name,
        layer=src.layer.name,
        f="CSV",
        spat=c(6, 48, 8, 50),
        overwrite=TRUE)
}
round(file.info(dst.target.name)$size/1024,1)


## ----import.profiles.csv-------------------------------------------------------------------------
layer.name <- "wosis_latest_profiles_ceu"
system.time(
  profiles.ceu <- read.csv(paste0(wosis.dir.name.ceu, "/",layer.name,".csv"),
                stringsAsFactors = FALSE)
)
names(profiles.ceu)


## ----download.bd.csv-----------------------------------------------------------------------------
src.layer.name <- "wosis_latest_bdfi33"
dst.layer.name <- "wosis_latest_bdfi33"
(dst.target.name <- paste0(wosis.dir.name,"/",dst.layer.name,".csv"))
if (!file.exists(dst.target.name)) { 
gdalUtilities::ogr2ogr(src=wfs,
        dst=dst.target.name,
        layer=src.layer.name,
        f="CSV",
        overwrite=TRUE)
}
file.info(dst.target.name)$size/1024/1024


## ----import.bd.csv-------------------------------------------------------------------------------
layer.name <- "wosis_latest_bdfi33"
system.time(
  bd33.pts <- read.csv(file=paste0(wosis.dir.name,"/",layer.name,".csv"),
                stringsAsFactors = FALSE)
)
dim(bd33.pts)
names(bd33.pts)



In [ ]:

## ----coordinates.profiles------------------------------------------------------------------------
profiles.ceu <- st_as_sf(profiles.ceu, 
                         coords = c("longitude", "latitude"),
                         crs = 4326)
class(profiles.ceu)


## ----profiles.wrb--------------------------------------------------------------------------------
table(profiles.ceu$cwrb_reference_soil_group)


In [ ]:


## ----map.profiles.wrb----------------------------------------------------------------------------
ggplot(data=profiles.ceu) +
  aes(col=cwrb_reference_soil_group) +
  geom_sf()



In [ ]:

## ----load.aqp------------------------------------------------------------------------------------
require(data.table)
require(aqp)            # Algorithms for Quantitative Pedology



In [ ]:

## ----bd.aqp--------------------------------------------------------------------------------------
ds.aqp <- st_drop_geometry(bd33)
depths(ds.aqp) <- profile_id ~ upper_dept + lower_dept
is(ds.aqp)
slotNames(ds.aqp)
str(ds.aqp@site)
str(ds.aqp@horizons)
head(ds.aqp@site)
head(ds.aqp@horizons[c(2,5,6,7,9)],12)


In [ ]:


## ----plot.bd.spc,fig.width=12, fig.height=8------------------------------------------------------
plotSPC(ds.aqp[1:24,], name="layer_name", color='bdfi33_v_1')

